In [7]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import underthesea #pip install underthesea

Path=os.getcwd() + "\\Data"

In [8]:
# Đọc file dữ liệu
def read_file(filename):	    
    filename=os.path.join(Path, filename)        
    with open(filename, 'r',encoding="utf-8") as f:
        data = f.read()

    return data.encode('utf-8')

# Chuẫn hóa dữ liệu
def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+4}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

# Tiền xử lý văn bản
import re
def Text_Preprocessing(doc):
    text_pre = doc.lower()
    
    text_pre = underthesea.word_tokenize(text_pre, format="text")
    text_pre = underthesea.text_normalize(text_pre)
    
    text_pre = text_pre.replace('XXXX', '')        
    text_pre = text_pre.replace(u'\ufffd', '')   # Replaces the ASCII symbol with ''
    text_pre = re.sub(r'[^\w\s]','',text_pre)
    text_pre = text_pre.rstrip('\n')              # Removes line breaks
    text_pre = text_pre.casefold()                # Makes all letters lowercase
    
    text_pre = re.sub('\W_',' ', text_pre)        # removes specials characters and leaves only words
    text_pre = re.sub("\S*\d\S*"," ", text_pre)   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
    text_pre = re.sub("\S*@\S*\s?"," ", text_pre) # removes emails and mentions (words with @)
    text_pre = re.sub(r'http\S+', '', text_pre)   # removes URLs with http
    text_pre = re.sub(r'www\S+', '', text_pre)    # removes URLs with www
        
        
    ##Remove stop words
    filename=os.path.join(Path, "vietnamese-stopwords.txt")
    f = open(filename, "r", encoding="utf-8")
    
    #Get Stop words Dictionaries
    List_StopWords=f.read().split("\n")
    text_pre=" ".join(text for text in text_pre.split() if text not in List_StopWords)
    
    return text_pre

In [9]:
# Define the Data
doc_4=read_file("doc4.txt")	#Crawl from: https://vi.wikipedia.org/wiki/X%E1%BB%AD_l%C3%BD_ng%C3%B4n_ng%E1%BB%AF_t%E1%BB%B1_nhi%C3%AAn
doc_5=read_file("doc5.txt") #Crawl from: https://en.wikipedia.org/wiki/Manchester_United_F.C.
doc_6=read_file("doc6.txt") #Crawl from: https://vi.wikipedia.org/wiki/Arsenal_F.C.

doc_4=Text_Preprocessing(doc_4)
doc_5=Text_Preprocessing(doc_5)
doc_6=Text_Preprocessing(doc_6)

data = [doc_4, doc_5, doc_6]
data

['xử_lý ngôn_ngữ tự_nhiên natural language processing nlp nhánh trí_tuệ nhân_tạo tập_trung ứng_dụng ngôn_ngữ con_người trí_tuệ nhân_tạo xử_lý ngôn_ngữ tự_nhiên liên_quan ý_nghĩa ngôn ngữcông cụ hoàn_hảo tư_duy giao_tiếp xử_lý phân_tích hình_thái phân_tích ký chữ dấu_câu tách tiếng ngôn_ngữ phân_tách dấu tuy_nhiên tiếng việt dấu phân_tách tiếng âm_tiết ngôn_ngữ tiếng trung tiếng hàn tiếng nhật phân_tách tiếng việt công_việc hề đơn_giản phân_tích cú_pháp dãy biến_đổi thành cấu_trúc thể_hiện liên_kết dãy vi_phạm luật văn_phạm phân_tích ngữ_nghĩa ngữ_nghĩa cấu_trúc phân_tích cú_pháp tích_hợp văn_bản ngữ_nghĩa câu riêng_biệt có_thể phụ_thuộc câu đứng đồng_thời có_thể ảnh_hưởng câu phân_tích thực_nghĩa cấu_trúc thể_hiện phát_ngôn thông_dịch xác_định thật_sự nghĩa tuy_nhiên ranh_giới xử_lý mong_manh có_thể tiến_hành bước_một tiến_hành tùy giải_thuật ngữ_cảnh cụ_thể bài_toán ứng_dụng nhận_dạng chữ viết hai kiểu nhận_dạng nhận_dạng chữ in ví_dụ nhận_dạng chữ sách_giáo_khoa thành dạng văn_bản đi

In [10]:
# Xây dựng vector TF-IDF
CountVect = CountVectorizer()
vector_matrix = CountVect.fit_transform(data)
vector_matrix.shape

(3, 941)

In [11]:
tokens = CountVect.get_feature_names_out()
create_dataframe(vector_matrix.toarray(),tokens)

# Tính toán độ tương đồng
cosine_similarity_matrix = cosine_similarity(vector_matrix)
Similarity=create_dataframe(cosine_similarity_matrix,['doc_4','doc_5','doc_6'])

Similarity

,doc_4,doc_5,doc_6
doc_4,1.000000,0.024140,0.034699
doc_5,0.024140,1.000000,0.712999
doc_6,0.034699,0.712999,1.000000
